In [0]:
SET use_cached_result = false;
SELECT  SUM(ss_net_profit) / SUM(ss_ext_sales_price) AS gross_margin
        ,i_category
        ,i_class
        ,grouping(i_category) + grouping(i_class) AS lochierarchy
        ,RANK() OVER (PARTITION BY grouping(i_category) + grouping(i_class),CASE   WHEN grouping(i_class) = 0 THEN i_category
        END ORDER BY SUM(ss_net_profit) / SUM(ss_ext_sales_price) ASC ) AS rank_within_parent
FROM workspace.tpcds_sf1000_iceberg.store_sales
,       workspace.tpcds_sf1000_iceberg.date_dim d1
,       workspace.tpcds_sf1000_iceberg.item
,       workspace.tpcds_sf1000_iceberg.store
WHERE   d1.d_year = 1999
AND     d1.d_date_sk = ss_sold_date_sk
AND     i_item_sk = ss_item_sk
AND     s_store_sk = ss_store_sk
AND     s_state IN ('MI','LA','NY','AL','GA','OH','NM','SD')
GROUP BY ROLLUP (i_category,i_class)
ORDER BY lochierarchy DESC,CASE   WHEN lochierarchy = 0 THEN i_category
END,rank_within_parent
LIMIT   100
;